# Variant Calling
Call genetic variants in the evolved line using genome assembly based on the ancestor line

<hr >

## Current Directory Structure

In [1]:
%%bash
cd ./analysis
ls -1F

assembly/
data/
fastqc-analysis/
mappings/
trimmed/


- data: Raw FASTQ files
- trimmed: Sickle trimmed FASTQ files
- fastqc-analysis: FASTQC analysis of raw and trimmed FASTQ files
- assembly: reference genome assembly from ancestral genome with bowtie and bwa indexed references
- mappings: bowtie and bwa aligned mappings (sorted and post-processed)

<hr >

## Install necessary tools with conda
- samtools: 1.9
- bcftools: 1.9
- bamtools: 2.5.1
- freebayes: 1.2.0
- vcflib: 1.0.0_rc2 
- rtg-tools: 3.10

<hr >

## Pre-process: Indexing
- Create index FASTA reference for SNP caller using SAMtools
- Create index BAM files using BAMtools

- Creates .fai index file in the assebly/bwa directory

- Creates .bai index file in the mappings/bwa directory

<hr >

## VCF file Format
| Column        | Field         | Description  |
| ------------- |:-------------:|:-----:|
| 1 |CHROM  |Chromosome name  |
| 2 |POS   |1-based position. For an indel, this is the position preceding the indel |
| 3 |ID  |Variant identifier (example: the dbSNP rsID) |
| 4 |REF    |Reference sequence at POS involved in the variant. For a SNP, it is a single base |
| 5 |ALT   | Comma delimited list of alternative seuqence(s)  |
| 6 |QUAL  |Phred-scaled probability of all samples being homozygous reference  |
| 7 |FILTER   |Semicolon delimited list of filters that the variant fails to pass |
| 8 |INFO   |Semicolon delimited list of variant information  |
| 9 |FORMAT  |Colon delimited list of the format of individual genotypes in the following fields  |
| 10|SAMPLES    |Individual genotype information defined by FORMAT |


<hr >

## Varinat Calling with Samtools
- Pile up all the reads with SAMtools mpileup:
    - -u: uncompressed output
    - -g: generate genotype likelihoods in BCF format
    - -f FILE: faidx indexed reference sequence file
- Call Variants with Bcftools call:
    - -v: output variant sites only
    - -m: alternative model for multiallelic and rare-variant calling
    - -o: output file-name
    - -O z: output type: ‘z’ compressed VCF
- Save output into variants directory

#### Count number of variants:

In [5]:
%%bash
zcat analysis/variants/evolved-6.mpileup.vcf.gz | grep -v '^#' | wc -l

695


<hr >

## Varinat Calling with Freebayes
- Reference genome scaffold file:
    - in fasta-format and the index in .fai format 
- Mapping BAM file:
    - Mapping file (.bam file) and a mapping index (.bai file)
- Callvariants with freebayes and pipe results to a new file
- -f --fasta-reference FILE: 
    - Use FILE as the reference sequence for analysis.  An index file (FILE.fai) will be created if none exists.  If neither --targets nor --region are specified, FreeBayes will analyze every position in this reference.


#### Count number of variants:

In [6]:
%%bash
zcat analysis/variants/evolved-6.freebayes.vcf.gz | grep -v '^#' | wc -l

5656


<hr >

## Index VCF files with tabix (SAMtools) to prepare vcf-file for querying
- tabix version: 1.9
- -p vcf: input format

- Creates .tbi index file

## Quick stats with rtg vcfstats (mpileup):

In [7]:
%%bash
rtg vcfstats analysis/variants/evolved-6.mpileup.vcf.gz

Location                     : analysis/variants/evolved-6.mpileup.vcf.gz
Failed Filters               : 0
Passed Filters               : 695
SNPs                         : 644
MNPs                         : 0
Insertions                   : 25
Deletions                    : 26
Indels                       : 0
Same as reference            : 0
SNP Transitions/Transversions: 1.64 (421/256)
Total Het/Hom ratio          : 15.95 (654/41)
SNP Het/Hom ratio            : 19.77 (613/31)
MNP Het/Hom ratio            : - (0/0)
Insertion Het/Hom ratio      : 2.13 (17/8)
Deletion Het/Hom ratio       : 12.00 (24/2)
Indel Het/Hom ratio          : - (0/0)
Insertion/Deletion ratio     : 0.96 (25/26)
Indel/SNP+MNP ratio          : 0.08 (51/644)


## Quick stats with rtg vcfstats (freebayes):

In [8]:
%%bash
rtg vcfstats analysis/variants/evolved-6.freebayes.vcf.gz

Location                     : analysis/variants/evolved-6.freebayes.vcf.gz
Failed Filters               : 0
Passed Filters               : 5656
SNPs                         : 739
MNPs                         : 151
Insertions                   : 42
Deletions                    : 70
Indels                       : 8
Same as reference            : 4646
SNP Transitions/Transversions: 1.25 (416/333)
Total Het/Hom ratio          : 47.10 (989/21)
SNP Het/Hom ratio            : 72.90 (729/10)
MNP Het/Hom ratio            : 74.50 (149/2)
Insertion Het/Hom ratio      : 7.40 (37/5)
Deletion Het/Hom ratio       : 22.33 (67/3)
Indel Het/Hom ratio          : 7.00 (7/1)
Insertion/Deletion ratio     : 0.60 (42/70)
Indel/SNP+MNP ratio          : 0.13 (120/890)


- More varinats called with Freebayes. 151 Multiple Nucleotide Polymorphisms compared to 0 with mpileup.   

## Statistics with BCFTools
- -s -: list of samples for sample stats, “-” to include all samples
- -F FILE: faidx indexed reference sequence file to determine INDEL context